<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [VAE7] - Variational AutoEncoder (VAE) with CelebA (medium)
<!-- DESC --> Episode 7 : Variational AutoEncoder (VAE) with CelebA (medium res.)
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Build and train a VAE model with a large dataset in **medium resolution(>140 GB)**
 - Understanding a more advanced programming model with **data generator**

The [CelebFaces Attributes Dataset (CelebA)](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) contains about 200,000 images (202599,218,178,3).  

## What we're going to do :

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard

## Step 1 - Setup environment
### 1.1 - Python stuff

In [1]:
import tensorflow as tf
import numpy as np
import os,sys
from importlib import reload

import modules.vae
import modules.data_generator

reload(modules.data_generator)
reload(modules.vae)

from modules.vae  import VariationalAutoencoder
from modules.data_generator import DataGenerator

sys.path.append('..')
import fidle.pwk_ns as ooo

place, datasets_dir = ooo.init()

VariationalAutoencoder.about()
DataGenerator.about()


FIDLE 2020 - Practical Work Module
Version              : 0.57 DEV
Run time             : Sunday 13 September 2020, 15:27:04
TensorFlow version   : 2.0.0
Keras version        : 2.2.4-tf
Current place        : Fidle at IDRIS
Datasets dir         : /gpfswork/rech/mlh/commun/datasets
Update keras cache   : Done

FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.28

FIDLE 2020 - DataGenerator
Version              : 0.4.1


### 1.2 - The good place

In [2]:
train_dir    = f'{datasets_dir}/celeba/clusters-m.train'
test_dir     = f'{datasets_dir}/celeba/clusters-m.test'

## Step 2 - DataGenerator and validation data
Ok, everything's perfect, now let's instantiate our generator for the entire dataset.

In [3]:
data_gen = DataGenerator(train_dir, 32, k_size=1)
x_test   = np.load(f'{test_dir}/images-000.npy')

print(f'Data generator : {len(data_gen)} batchs of {data_gen.batch_size} images, or {data_gen.dataset_size} images')
print(f'x_test : {len(x_test)} images')

Data generator : 6250 batchs of 32 images, or 200000 images
x_test : 2599 images


## Step 3 - Get VAE model

In [4]:
tag = f'CelebA.002-M.{os.getenv("SLURM_JOB_ID","unknown")}'

input_shape = (192, 160, 3)
z_dim       = 200
verbose     = 0

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':3,  'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/CelebA.002-M.266176
Config saved in     : ./run/CelebA.002-M.266176/models/vae_config.json


## Step 4 - Compile it

In [5]:
optimizer     = tf.keras.optimizers.Adam(1e-4)
r_loss_factor = 10000

vae.compile(optimizer, r_loss_factor)

Compiled.


## Step 5 - Train
For 10 epochs, adam optimizer :  
- Run time at IDRIS : 1299.77 sec. - 0:21:39
- Run time at GRICAD : 2092.77 sec. - 0:34:52
- Run time at IDRIS with medium resolution : Train duration : 6638.61 sec. - 1:50:38

In [6]:
epochs            = 20
initial_epoch     = 0

In [7]:
vae.train(data_generator    = data_gen,
          x_test            = x_test,
          epochs            = epochs,
          initial_epoch     = initial_epoch
         )

Epoch 1/20
6250/6250 [==============================] - 357s 57ms/step - loss: 334.6058 - vae_r_loss: 286.9088 - vae_kl_loss: 47.6970 - val_loss: 241.3782 - val_vae_r_loss: 194.3746 - val_vae_kl_loss: 47.1940
Epoch 2/20
6250/6250 [==============================] - 361s 58ms/step - loss: 233.6622 - vae_r_loss: 178.5938 - vae_kl_loss: 55.0685 - val_loss: 223.6776 - val_vae_r_loss: 171.8392 - val_vae_kl_loss: 51.9847
Epoch 3/20
6250/6250 [==============================] - 362s 58ms/step - loss: 223.8023 - vae_r_loss: 166.8294 - vae_kl_loss: 56.9731 - val_loss: 223.1821 - val_vae_r_loss: 171.0302 - val_vae_kl_loss: 52.3176
Epoch 4/20
6250/6250 [==============================] - 361s 58ms/step - loss: 212.7579 - vae_r_loss: 154.4291 - vae_kl_loss: 58.3290 - val_loss: 207.5755 - val_vae_r_loss: 151.5950 - val_vae_kl_loss: 56.0710
Epoch 8/20
5728/6250 [==========================>...] - ETA: 29s - loss: 207.6788 - vae_r_loss: 149.3138 - vae_kl_loss: 58.3651

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>